### Problem 8 hetmanów 


[Wikipedia](https://pl.wikipedia.org/wiki/Problem_o%C5%9Bmiu_hetman%C3%B3w)  
<p>Problem ośmiu hetmanów – hetman jest figurą szachową, która bije figury znajdujące się w tej samej kolumnie, wierszu lub przekątnej, co on sam.</p>
<p>W jaki sposób rozstawić osiem hetmanów na tradycyjnej szachownicy 8x8 tak, aby wzajemnie się nie atakowały? Ile jest możliwych rozstawień?</p>
<p>Przez rozstawienie podstawowe bądź rozwiązanie podstawowe należy rozumieć rozwiązanie z dokładnością do izomorfizmu, tzn. z uwzględnieniem wszystkich pokrewnych pozycji wynikających z odbić zwierciadlanych i obrotów.</p>

#### Zakres ruchów hetmana
![Hetman](queen.png "Hetman i jego ruchy")


**Genotyp** to sposób kodowania części problemu, dzięki czemu można nimi manipulować za pomocą algorytmu genetycznego.  
Przykład: potencjalne genotypy dla tego problemu:  
  * 64 bity, po jednym dla każdego z 64 kwadratów na planszy 
  * 48 bitów, po 6 dla każdej z lokalizacjihetmana, ponieważ możemy policzyć do 64 za pomocą 6 bitów  
  * 8 liczb całkowitych z zakresu 0..63 lub 1..64  
  * 16 liczb całkowitych reprezentujących położenie wiersza i kolumny każdego hetmana  
  
**Fenotyp** mówi nam, jak odkodowane geny są wykorzystywane do rozwiązania problemu.  
W każdym z powyższych przykładów fenotyp to lokalizacje 8 hetmanów na planszy.  

**Funkcja sprawności (ang. fitness)** ocenia następnie fenotyp w kontekście rozwiązania problemu w celu obliczenia  wartości funkcji sprawności.